In [138]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import scipy
import math
%matplotlib inline

In [139]:
est = pd.read_excel('delphi-consensus-outputs.xlsx')
rev = pd.read_excel('margin-revenue-salesforce.xlsx')

In [140]:
est

,Unnamed: 0,Naprosyn,Nipro,Anaprox,Norinyl,Pironil,Lidex,Synalar,Nasalide
0,Zero,0.30,0.47,0.15,0.31,0.45,0.56,0.59,0.15
1,50% of current,0.45,0.68,0.48,0.63,0.70,0.80,0.76,0.61
2,100% of current,1.00,1.05,1.04,1.03,1.01,1.02,1.02,1.07
3,150% of current,1.40,1.26,1.20,1.15,1.05,1.11,1.07,1.46
4,Saturation,1.60,1.52,1.35,1.25,1.10,1.20,1.11,1.76


In [141]:
rev = rev.drop(' ', axis =1)

In [142]:
adbug_min = est.min()[1:]

In [143]:
adbug_max = est.max()[1:]

### Part A:

In [144]:
from scipy.optimize import curve_fit

def func(SF, adbug_min, adbug_max, c,d):
    return adbug_min + (adbug_max-adbug_min)*((SF**c)/(d + SF**c))
p0 = np.zeros((8,4))
SF = [0,0.5,1,1.5,10]
for i in range(8):
    p0[i] = [1,1,adbug_min[i],adbug_max[i]]

response = est[est.columns[1:]]
popt = []
cov = []
for i in range(8):
    popt.append(curve_fit(func, SF, response.iloc[:,i], p0[i])[0])
    cov.append(curve_fit(func, SF, response.iloc[:,i], p0[i])[1])

In [145]:
nonlin = pd.DataFrame(popt,columns=['Min','Max','c','d'],index =adbug_max.index )
nonlin

,Min,Max,c,d
Naprosyn,0.31,1.61,3.46,0.84
Nipro,0.47,1.52,2.26,0.83
Anaprox,0.15,1.34,2.82,0.36
Norinyl,0.31,1.25,2.62,0.31
Pironil,0.45,1.09,3.25,0.16
Lidex,0.56,1.20,2.09,0.39
Synalar,0.59,1.11,3.25,0.21
Nasalide,0.16,1.78,2.05,0.68


### Part B:

In [146]:
rev1 = np.asarray(rev).transpose()
nonlin1 = np.asarray(nonlin)
rev1

array([[  0.7 , 214.4 ,  96.8 ],
       [  0.8 , 210.1 ,  97.2 ],
       [  0.55,  36.5 , 142.4 ],
       [  0.72,  21.2 ,  52.7 ],
       [  0.72,  37.2 ,  24.1 ],
       [  0.62,  38.  ,  27.3 ],
       [  0.53,  14.6 ,  29.7 ],
       [  0.52,  11.2 ,  56.8 ]])

In [174]:
def negative_profit(x0):
    resp = func((x0/rev1[i][2]),nonlin1[i][0],nonlin1[i][1],nonlin1[i][2],nonlin1[i][3])
    profit = (resp*rev.loc[1][i]*rev.loc[0][i]) - (x0*0.057)
    return (-1*profit)

In [175]:
import scipy.optimize as optimize
import numpy as np
lower_bound = 0
x0 = 80 
bounds_object = optimize.Bounds(lower_bound, np.inf)
opt_sales = np.zeros(8)
for i in range(8):
    optimizer_output = optimize.minimize(negative_profit,x0, method='trust-constr', bounds=bounds_object, options={'verbose': 1})
    opt_sales[i] = optimizer_output.x

`gtol` termination condition is satisfied.
Number of iterations: 25, function evaluations: 30, CG iterations: 14, optimality: 7.09e-09, constraint violation: 0.00e+00, execution time: 0.076 s.
`xtol` termination condition is satisfied.
Number of iterations: 53, function evaluations: 40, CG iterations: 41, optimality: 1.73e-08, constraint violation: 0.00e+00, execution time: 0.094 s.
`gtol` termination condition is satisfied.
Number of iterations: 18, function evaluations: 20, CG iterations: 9, optimality: 2.88e-09, constraint violation: 0.00e+00, execution time: 0.041 s.
`gtol` termination condition is satisfied.
Number of iterations: 15, function evaluations: 14, CG iterations: 6, optimality: 3.10e-09, constraint violation: 0.00e+00, execution time: 0.032 s.


/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '
/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


`gtol` termination condition is satisfied.
Number of iterations: 22, function evaluations: 30, CG iterations: 14, optimality: 2.38e-09, constraint violation: 0.00e+00, execution time: 0.051 s.
`gtol` termination condition is satisfied.
Number of iterations: 22, function evaluations: 24, CG iterations: 11, optimality: 9.74e-09, constraint violation: 0.00e+00, execution time: 0.067 s.
`gtol` termination condition is satisfied.
Number of iterations: 20, function evaluations: 26, CG iterations: 12, optimality: 8.69e-09, constraint violation: 0.00e+00, execution time: 0.052 s.
`gtol` termination condition is satisfied.
Number of iterations: 13, function evaluations: 12, CG iterations: 5, optimality: 7.93e-09, constraint violation: 0.00e+00, execution time: 0.032 s.


In [193]:
opt_fun = []
for i in range(8):
    opt_fun.append(-negative_profit(opt_sales[i]))
df = np.transpose([opt_sales,opt_fun])

In [194]:
df[:,0]

array([270.5127472 , 330.04559463, 171.68612697,  71.33334766,
        36.88030477,  50.00488776,  30.65498601,  71.1607629 ])

In [165]:
pd.DataFrame(df,columns = ['Optimal Sales Force','Objective function Value'],index =adbug_max.index)

,Optimal Sales Force,Objective function Value
Naprosyn,270.51,221.19
Nipro,330.05,228.62
Anaprox,171.69,12.95
Norinyl,71.33,13.21
Pironil,36.88,26.47
Lidex,50.00,23.96
Synalar,30.65,6.17
Nasalide,71.16,3.49


### Part C:

In [195]:
def negative_profit2(NSF):
    total_profit = 0
    prof = np.zeros(8)
    for i in range(8):
        resp = func((NSF[i]/rev1[i][2]),nonlin1[i][0],nonlin1[i][1],nonlin1[i][2],nonlin1[i][3])
        profit = (resp*rev1[i][1]*rev1[i][0]) - (NSF[i]*0.057)
        total_profit += profit
        prof[i] = profit 
    return -1*total_profit

In [196]:
n_drugs = 8
total_salesforce_size = 700
lower_bound = 0
x0 = np.ones(n_drugs)*total_salesforce_size/n_drugs
sum_constraint_object = optimize.LinearConstraint(np.ones((1,n_drugs)), lower_bound, total_salesforce_size)
bounds_object = optimize.Bounds(lower_bound, np.inf)
opt_sales = np.zeros(8)
optimizer_output = optimize.minimize(negative_profit2, x0, method='SLSQP', bounds=bounds_object, constraints=sum_constraint_object)
opt_sales = optimizer_output.x

In [209]:
Objective_func = []
for i in range(8):
    Objective_func.append(-negative_profit(opt_sales[i]))
percentprofit_red = -((Objective_func - df[:,1])/df[:,1])*100
percent_red = -(((opt_sales - df[:,0])/df[:,0]))*100
lf = [opt_sales,Objective_func,percentprofit_red,percent_red]
lf = pd.DataFrame(np.transpose(lf), columns = ['Optimal Sales Force','Objective function Value','Percent Profit Reduction','Percent Salesforce Reduction'],index =adbug_max.index)

pd.options.display.float_format = '{:.2f}'.format

In [210]:
(opt_sales - df[:,0])/df[:,0]

array([-0.18577901, -0.25487989, -0.31589005, -0.27759557, -0.20034839,
       -0.29318662, -1.        , -1.        ])

In [211]:
lf

,Optimal Sales Force,Objective function Value,Percent Profit Reduction,Percent Salesforce Reduction
Naprosyn,220.26,219.55,0.74,18.58
Nipro,245.92,225.90,1.19,25.49
Anaprox,117.45,11.07,14.54,31.59
Norinyl,51.53,12.55,4.99,27.76
Pironil,29.49,26.23,0.92,20.03
Lidex,35.34,23.48,1.99,29.32
Synalar,0.00,4.56,26.03,100.00
Nasalide,0.00,0.91,73.79,100.00


In [212]:
rev

,Naprosyn,Nipro,Anaprox,Norinyl,Pironil,Lidex,Synalar,Nasalide
0,0.70,0.80,0.55,0.72,0.72,0.62,0.53,0.52
1,214.40,210.10,36.50,21.20,37.20,38.00,14.60,11.20
2,96.80,97.20,142.40,52.70,24.10,27.30,29.70,56.80


In [135]:
est

,Unnamed: 0,Naprosyn,Nipro,Anaprox,Norinyl,Pironil,Lidex,Synalar,Nasalide
0,Zero,0.30,0.47,0.15,0.31,0.45,0.56,0.59,0.15
1,50% of current,0.45,0.68,0.48,0.63,0.70,0.80,0.76,0.61
2,100% of current,1.00,1.05,1.04,1.03,1.01,1.02,1.02,1.07
3,150% of current,1.40,1.26,1.20,1.15,1.05,1.11,1.07,1.46
4,Saturation,1.60,1.52,1.35,1.25,1.10,1.20,1.11,1.76


- Naprosyn: had the least change salesforce which makes sense because it was generating the most revenue and had an average profit margin. Also, after 100% revenue still tends to change by 60% before reaching saturation.
- Nipro: had the the least change in salesforce which makes sense because it is the most profitable and generates a lot of revenue. 
- Synalar & Nasalide: had the highest % reduction in salesforce which makes sense because they had the lowest profit margin and generated the least revenue.These drugs reach saturation quite early when compared to the others. 
- Pironil,Norinyl & Anaprox: had a larger decrease w.r.t other drugs in this category although it was significant. This is because these drug tend to reach saturation earlier. 
- Lidex: had a small change in salesforce compared to other drugs. This is most likely because it was performing well "as-is".